# Lấy thông tin khoá học


- Tư duy như sau: 
 + Cần lấy log của khoá học
 + Cần lấy cấu trúc, thông tin khoá học
-> API get khoá học
-> API get danh sách thành viên trong khoá học (giáo viên và học sinh)
-> API log của từng người trong khoá học

-> Hoàn thiện: json_course_moodle.json -> json_course_moodle_simple.json -> json_course_moodle_hierarchy_clean.json

In [3]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML
from bs4 import BeautifulSoup

AttributeError: partially initialized module 'pandas' has no attribute '_pandas_parser_CAPI' (most likely due to a circular import)

In [4]:


# Cấu hình Moodle API
moodle_url = "http://localhost:8100/webservice/rest/server.php"
api_token = "873fe06ae8e762265a72049a30cfbdc4" #token của webservice moodle

def call_moodle_api(wsfunction, params=None):
    """
    Gọi Moodle Web Service API với hàm và tham số được chỉ định.

    Args:
        wsfunction (str): Tên hàm Web Service.
        params (dict, optional): Tham số bổ sung cho API.

    Returns:
        dict or list: Dữ liệu JSON trả về từ API hoặc None nếu có lỗi.
    """
    default_params = {
        'wstoken': api_token,
        'wsfunction': wsfunction,
        'moodlewsrestformat': 'json'
    }
    if params:
        default_params.update(params)
    
    try:
        response = requests.get(moodle_url, params=default_params)
        if response.status_code != 200:
            print(f"❌ Lỗi HTTP: {response.status_code}")
            print("Nội dung phản hồi:", response.text)
            return None
        
        data = response.json()
        if isinstance(data, dict) and 'exception' in data:
            print(f"❌ Lỗi API: {data['errorcode']}")
            print(f"Thông điệp: {data['message']}")
            return None
        
        return data
    
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi khi gửi yêu cầu: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ Lỗi phân tích JSON: {e}")
        print("Nội dung phản hồi:", response.text)
        return None

def get_all_courses():
    """
    Lấy danh sách tất cả các khóa học.

    Returns:
        list: Danh sách các khóa học hoặc None nếu có lỗi.
    """
    data = call_moodle_api('core_course_get_courses')
    return data if data else []

def get_course_contents(courseid):
    """
    Lấy chi tiết nội dung của một khóa học, bao gồm section và module.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách các section hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_course_get_contents', {'courseid': courseid})

def get_lesson_details(lessonid):
    """
    Lấy chi tiết một bài học (lesson).

    Args:
        lessonid (int): ID của bài học.

    Returns:
        dict: Thông tin bài học hoặc None nếu có lỗi.
    """
    return call_moodle_api('mod_lesson_get_lesson', {'lessonid': lessonid})

def get_lesson_pages(lessonid):
    """
    Lấy danh sách các trang trong một bài học.

    Args:
        lessonid (int): ID của bài học.

    Returns:
        list: Danh sách các trang hoặc None nếu có lỗi.
    """
    return call_moodle_api('mod_lesson_get_pages', {'lessonid': lessonid})

def get_list_of_users_in_course(courseid):
    """
    Lấy danh sách người dùng trong một khóa học.

    Args:
        courseid (int): ID của khóa học.

    Returns:
        list: Danh sách người dùng hoặc None nếu có lỗi.
    """
    return call_moodle_api('core_enrol_get_enrolled_users', {'courseid': courseid})


### Danh sách người tham gia khoá học (cả giáo viên và học sinh)

In [6]:
courseid = 5
users = get_list_of_users_in_course(courseid)
if users:
    for user in users:
        print(user);
        #  print(f"ID: {user['id']}, Tên: {user['fullname']}, Email: {user['email']}") 

{'id': 2, 'username': 'admin', 'firstname': 'Nguyen', 'lastname': 'Loc', 'fullname': 'Nguyen Loc', 'email': 'lockbkbang@gmail.com', 'department': '', 'firstaccess': 1750565372, 'lastaccess': 1755658468, 'lastcourseaccess': 1755581573, 'description': '', 'descriptionformat': 1, 'country': 'VN', 'profileimageurlsmall': 'http://localhost:8100/theme/image.php?theme=boost&component=core&rev=1755656823&image=u%2Ff2', 'profileimageurl': 'http://localhost:8100/theme/image.php?theme=boost&component=core&rev=1755656823&image=u%2Ff1', 'roles': [{'roleid': 3, 'name': '', 'shortname': 'editingteacher', 'sortorder': 0}], 'preferences': [{'name': 'auth_manual_passwordupdatetime', 'value': '1750565447'}, {'name': 'block_myoverview_user_grouping_preference', 'value': 'all'}, {'name': 'core_message_migrate_data', 'value': '1'}, {'name': 'core_user_welcome', 'value': '1750565473'}, {'name': 'coursesectionspreferences_4', 'value': '{"contentcollapsed":{"2":25,"5":26,"6":27,"8":29,"9":30,"10":31,"14":24},"

### Lấy tất cả khoá học


In [5]:
get_all_courses()

[{'id': 1,
  'shortname': 'Loc',
  'categoryid': 0,
  'categorysortorder': 1,
  'fullname': 'Nguyen Huu Loc',
  'displayname': 'Nguyen Huu Loc',
  'idnumber': '',
  'summary': '',
  'summaryformat': 0,
  'format': 'site',
  'showgrades': 1,
  'newsitems': 3,
  'startdate': 0,
  'enddate': 0,
  'numsections': 1,
  'maxbytes': 0,
  'showreports': 0,
  'visible': 1,
  'groupmode': 0,
  'groupmodeforce': 0,
  'defaultgroupingid': 0,
  'timecreated': 1750565334,
  'timemodified': 1750565466,
  'enablecompletion': 0,
  'completionnotify': 0,
  'lang': '',
  'forcetheme': '',
  'courseformatoptions': [{'name': 'numsections', 'value': 1}],
  'showactivitydates': False,
  'showcompletionconditions': None},
 {'id': 4,
  'shortname': 'Java',
  'categoryid': 1,
  'categorysortorder': 10002,
  'fullname': 'NGÔN NGỮ LẬP TRÌNH JAVA',
  'displayname': 'NGÔN NGỮ LẬP TRÌNH JAVA',
  'idnumber': '',
  'summary': '',
  'summaryformat': 1,
  'format': 'topics',
  'showgrades': 1,
  'newsitems': 5,
  'startd

### Cấu trúc khoá học


In [15]:
import json

courseid = 5
data = get_course_contents(courseid)  # Giả sử hàm này đã trả về JSON từ Moodle

# Lưu vào file
with open("json_course_moodle.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2, ensure_ascii=False)

print("✅ Đã lưu JSON vào file 'json_course_moodle.json'")

✅ Đã lưu JSON vào file 'json_course_moodle.json'


In [16]:
import json

def build_hierarchical_structure(json_file):
    with open(json_file, 'r', encoding='utf-8') as f:
        course_data = json.load(f)

    course_structure = []

    def process_modules(modules):
        items = []
        for m in modules:
            mod_info = {
                'id': m.get('id'),
                'name': m.get('name'),
                'modname': m.get('modname')
            }
            items.append(mod_info)
        return items

    def process_sections(sections):
        hierarchy = []
        for sec in sections:
            sec_dict = {
                'sectionid': sec.get('id'),
                'name': sec.get('name'),
                'lessons': []
            }
            for mod in sec.get('modules', []):
                if mod.get('modname') == 'subsection':
                    lesson_dict = {
                        'sectionid': mod.get('id'),
                        'name': mod.get('name'),
                        'resources': process_modules(mod.get('modules', []))  # resource bên trong
                    }
                    sec_dict['lessons'].append(lesson_dict)
                else:
                    # Nếu trực tiếp là tài nguyên trong section
                    sec_dict.setdefault('resources', []).append({
                        'id': mod.get('id'),
                        'name': mod.get('name'),
                        'modname': mod.get('modname')
                    })
            hierarchy.append(sec_dict)
        return hierarchy

    course_structure = process_sections(course_data)
    return course_structure

# --- Demo sử dụng ---
course_hierarchy = build_hierarchical_structure('json_course_moodle.json')
# Lưu vào file
with open("json_course_moodle_simple.json", "w", encoding="utf-8") as f:
    json.dump(course_hierarchy, f, indent=2, ensure_ascii=False)

print("✅ Đã lưu JSON vào file 'json_course_moodle_simple.json'")
print(json.dumps(course_hierarchy, indent=2, ensure_ascii=False))

✅ Đã lưu JSON vào file 'json_course_moodle_simple.json'
[
  {
    "sectionid": 33,
    "name": "General",
    "lessons": [],
    "resources": [
      {
        "id": 52,
        "name": "Announcements",
        "modname": "forum"
      },
      {
        "id": 60,
        "name": "Tin 12 - Định hướng Khoa học Máy tính course question bank",
        "modname": "qbank"
      },
      {
        "id": 102,
        "name": "Bài kiểm tra cuối kỳ môn Tin học - 120825 - 10",
        "modname": "quiz"
      },
      {
        "id": 103,
        "name": "Bài kiểm tra cuối kỳ môn Tin học - 120825 - 11",
        "modname": "quiz"
      },
      {
        "id": 104,
        "name": "Bài kiểm tra cuối kỳ môn Tin học - 120825 - 20",
        "modname": "quiz"
      },
      {
        "id": 105,
        "name": "Bài kiểm tra cuối kỳ môn Tin học - 120825 - 200",
        "modname": "quiz"
      }
    ]
  },
  {
    "sectionid": 34,
    "name": "Chủ đề 1: MÁY TÍNH VÀ XÃ HỘI TRI THỨC",
    "lessons": [
   

In [2]:
import json

# Load JSON gốc
with open("json_course_moodle_simple.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Tạo map từ tên bài học sang resources thực tế
resources_map = {}
for section in data:
    if section.get("resources"):
        resources_map[section["name"]] = {
            "sectionIdNew": section["sectionid"],
            "resources": section["resources"]
        }

new_structure = []

for idx, section in enumerate(data):
    if idx == 0:
        # Giữ nguyên General như cũ
        new_structure.append(section)
        continue

    if section.get("lessons"):  # section có lessons => chủ đề
        new_section = {
            "sectionIdOld": section["sectionid"],
            "name": section["name"],
            "lessons": []
        }

        for lesson in section["lessons"]:
            res_info = resources_map.get(lesson["name"], {"sectionIdNew": lesson["sectionid"], "resources": []})
            new_section["lessons"].append({
                "sectionIdOld": lesson["sectionid"],
                "sectionIdNew": res_info["sectionIdNew"],
                "name": lesson["name"],
                "resources": res_info["resources"]
            })

        new_structure.append(new_section)

    # Bỏ các section riêng lẻ không thuộc chủ đề, ngoại trừ General

# Lưu JSON mới
with open("json_course_moodle_hierarchy_clean.json", "w", encoding="utf-8") as f:
    json.dump(new_structure, f, ensure_ascii=False, indent=2)

print("Đã tạo xong json_course_moodle_hierarchy_clean.json, giữ lại General, bỏ các section riêng lẻ cũ khác.")

Đã tạo xong json_course_moodle_hierarchy_clean.json, giữ lại General, bỏ các section riêng lẻ cũ khác.
